In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import requests
from pathlib import Path
import scipy as stats

In [ ]:
restaurantpath= Path("csv_folder/restaurant_data.csv")
restaurantfile=pd.read_csv(restaurantpath,encoding="UTF-8")
restaurantfile


In [ ]:
restaurantfile.columns

In [ ]:
 # Define subset of columns to consider for duplicates
restaurantfile['CAMIS'].value_counts()

In [ ]:
#Delete unnessecary columns
restaurants_to_keep=['CAMIS','BORO','DBA', 'ZIPCODE','ACTION','CUISINE DESCRIPTION','INSPECTION DATE','GRADE']
restaurantfile=restaurantfile[restaurants_to_keep].copy()
#restaruantfile = restaruantfile.drop(columns=['BUILDING','STREET', 'PHONE', 'Community Board', 'Council District', 'Census Tract', 'BIN', 'BBL', 'NTA', 'RECORD DATE', 'GRADE DATE'])
restaurantfile.head()

In [ ]:
#Drop restaurants that don't have a grade (i.e., NaN) from the dataframe
#query_df['grade'].unique()
clean_grade_df = restaurantfile.dropna(subset=['GRADE'])

In [ ]:
#getting unique value lists which should be helpful later when making calls. this syntax can work for any column.
clean_grade_df['CUISINE DESCRIPTION'].unique().tolist()
clean_grade_df['BORO'].unique().tolist()

In [ ]:
#the code worked but i think we will have to run the dates through that filter code to get it sorted. I am holdingoff until class to do this.
clean_grade_df.sort_values('INSPECTION DATE',ascending=False)
clean_grade_df.head()

In [ ]:
#Format dates, adds a date column, fills NaN, get rid of decimal in zip code
clean_grade_df['INSPECTION DATE'] = pd.to_datetime(clean_grade_df['INSPECTION DATE']).dt.date
clean_grade_df['Date']=clean_grade_df['INSPECTION DATE']
clean_grade_df['ZIPCODE'].fillna(-1,inplace=True)
clean_grade_df['ZIPCODE'] = clean_grade_df['ZIPCODE'].astype(int)
clean_grade_df

In [ ]:
#Need denominator to be unique restaurants
clean_grade_unique=clean_grade_df.drop_duplicates(subset=['CAMIS'])
clean_grade_unique.reset_index(drop=True, inplace=True)
clean_grade_unique

In [ ]:
# QUESTION 1: How many unique A, B, C restaurants were there in New York City from September 2010-September 2017?
gradecountscitywide=clean_grade_unique['GRADE'].value_counts()
gradecountscitywide

In [ ]:
#Grades by count pie chart:
explode = (0.1, 0.1, 0.2, 0.3,0.5)

#Pie chart
plt.pie(gradecountscitywide,explode=explode, labels='ABCPN',
        autopct="%1.1f%%", shadow=True, startangle=180)
plt.title('Grades Counts')
plt.axis("equal")
plt.show()

In [ ]:
#QUESTION 1a: Is there an association between resaurant cuisine and rating?
# Extract unique cuisine descriptions
cuisinedescriptions = clean_grade_unique['CUISINE DESCRIPTION'].unique()
cuisinedescriptions

In [ ]:
# Pare down list of cuisines for Chi-Square test

# Define a mapping dictionary for collapsing cuisines
cuisine_mapping = {
    'American': 'American',
    'Other': 'Other',
    'Korean': 'Asian',
    'Mexican': 'Latin American',
    'Latin American': 'Latin American',
    'Frozen Desserts': 'Desserts/Baked Goods',
    'Hotdogs': 'Fast Food',
    'Pizza': 'Pizza',
    'French':'European',
    'Bottled Beverages':'Coffee/Tea/Beverages',
    'Italian':'Italian',
    'Tex-Mex':'Latin American',
    'Japanese':'Asian',
    'Spanish':'European',
    'Coffee/Tea':'Coffee/Tea/Beverages',
    'Mediterranean':'European',
    'Caribbean':'Caribbean',
    'Middle Eastern': 'Middle Eastern',
    'German':'European',
    'Hamburgers':'Fast Food',
    'Sandwiches/Salads/Mixed Buffet':'Deli',
    'Vegan':'Special Diet',
    'Bagels/Pretzels':'Deli',
    'Jewish/Kosher':'Special Diet',
    'Filipino':'Asian',
    'Indian':'Indian',
    'Sandwiches':'Deli',
    'Seafood':'Seafood',
    'Chinese':'Asian',
    'Eastern European':'European',
    'Donuts':'Desserts/Baked Goods',
    'Soups/Salads/Sandwiches':'Deli',
    'Juice, Smoothies, Fruit Salads':'Juices/Smoothies',
    'African':'African',
    'Russian':'European',
    'Ethiopian':'African',
    'Mexican':'Latin American',
    'Steakhouse':'Steakhouse',
    'Irish':'European'
}

# Replace values in the 'CUISINES' column using the mapping dictionary
clean_grade_unique['CollapsedCUISINES'] = clean_grade_unique['CUISINE DESCRIPTION'].replace(cuisine_mapping)

cuisinedescriptionsnew = clean_grade_unique['CollapsedCUISINES'].unique()
cuisinedescriptionsnew

In [ ]:
# Is there an association between restaurant type and rating? -Chi-Square
from scipy.stats import chi2_contingency

chi2, p, dof, expected = chi2_contingency(pd.crosstab(clean_grade_unique['CollapsedCUISINES'], clean_grade_unique['GRADE']))

print("Chi-square test statistic:", chi2)
print(f"p-value: {p:.5f}")
print("Degress of Freedom:", dof)

cuisinebyratingdf=pd.crosstab(clean_grade_unique['CollapsedCUISINES'], clean_grade_unique['GRADE'])
cuisinebyratingdf

In [ ]:
# QUESTION 3: Number of grades by Borough among unique restaurants
gradecounts= clean_grade_unique.groupby(['BORO', 'GRADE']).size().reset_index(name='count')

Boroughs = gradecounts['BORO']
counts = gradecounts['count']
grades = gradecounts['GRADE']

transposecounts = gradecounts.pivot(index='BORO', columns='GRADE', values='count').fillna(0)

transposecounts.plot(kind='bar', stacked=True)
plt.xlabel('Borough')
plt.ylabel('Number of Restaurant Grades')
plt.title('Number of Restaurant Grades by Borough, 2010-2017')
plt.xticks(rotation=0) 
plt.tight_layout()
plt.show()

In [ ]:
# Is there an association between Borough and rating? -Chi-Square
from scipy.stats import chi2_contingency

chi2, p, dof, expected = chi2_contingency(pd.crosstab(clean_grade_unique['GRADE'], clean_grade_unique['BORO']))

print("Chi-square test statistic:", chi2)
print(f"p-value: {p:.5f}")
print("Degress of Freedom:", dof)

gradebyboro=pd.crosstab(clean_grade_unique['GRADE'], clean_grade_unique['BORO'])
gradebyboro


In [ ]:

crestaurants=clean_grade_unique.loc[clean_grade_unique['GRADE']=='C']
crestaurants
#italian_df=clean_grade_unique.loc[clean_grade_unique['CUISINE DESCRIPTION']=='Italian']
#italian_df.loc[italian_df['GRADE']=='C']

In [ ]:
actions=restaurantfile['ACTION'].unique()
actions

In [ ]:
closedrestaurants=restaurantfile.loc[restaurantfile['ACTION']=='Establishment Closed by DOHMH. Violations were cited in the following area(s) and those requiring immediate action were addressed.']
closedrestaurants

In [ ]:
#QUESTION 3A. Do certain boroughs have more restaurant violations? Violations per restaurant by borough
violationcount= clean_grade_df.groupby(['BORO']).size().reset_index(name='count')

camis_df1 = pd.DataFrame({
    'borough': ['Manhattan', 'Brooklyn', 'Queens', 'Bronx', 'Staten Island'],
    'Count': [104, 35, 159, 62, 12]
})

violation_df2 = pd.DataFrame({
    'borough': ['Manhattan', 'Brooklyn', 'Queens', 'Bronx', 'Staten Island'],
    'Count': [252, 91, 371, 119, 29]
})

#Merge the DataFrames on 'borough'
merged_df = pd.merge(camis_df1, violation_df2, on='borough', suffixes=('_restaurants', '_violations'))

#Perform the division
merged_df['Violations_per_Restaurant'] = merged_df['Count_violations'] / merged_df['Count_restaurants']

#Drop columns
merged_df.drop(['Count_restaurants', 'Count_violations'], axis=1, inplace=True)

#Display the result
print("\nMerged DataFrame with Division:")
print(merged_df)

merged_df.plot(kind='bar', x='borough', y='Violations_per_Restaurant')
plt.xlabel('Borough')
plt.ylabel('Number of Violations per Restaurant')
plt.title('Violations per Restaurant by Borough, New York City 2010-2017')
plt.xticks(rotation=0) 
plt.tight_layout()
plt.show()

BEGIN RAT DATA DO NOT PUT PIZZA DATA BELOW

In [ ]:
ratpath= Path("csv_folder/rat_sightings.csv")
ratfile=pd.read_csv(ratpath,encoding="UTF-8")
ratfile.head()

In [ ]:
ratfile.columns

In [ ]:
#came up with way to easily filter the main DF by columns. This can easily be amended by adding column titles to columns_to_keep and rerunning the block.
#i took facility type out Facililty Type because it is only Nans. scroll down to see output confirming.
columns_to_keep=['Borough','Incident Zip','Created Date','Location Type','City','Status','Complaint Type']
clean_rats_df=ratfile[columns_to_keep].copy()
clean_rats_df

In [ ]:
#fixed the zip code formatting, had to fill NaNs but we can discuss dropping them maybe? Im having trouble picturing how it will effect the final project.
#next we need to run that code on the date here, i cant seem to locate it
clean_rats_df['Incident Zip'].fillna(0,inplace=True)
clean_rats_df['Incident Zip'] = clean_rats_df['Incident Zip'].astype(int)
clean_rats_df['Created Date'] = pd.to_datetime(clean_rats_df['Created Date']).dt.date
clean_rats_df['Date']=clean_rats_df['Created Date']
clean_rats_df['ZIPCODE']=clean_rats_df['Incident Zip']
clean_rats_df

In [ ]:
#get the number of places closed due to violations by borough.
closed_df=clean_rats_df[clean_rats_df['Status']=='Closed'].groupby('Borough').size()
closed_df

In [ ]:
#bar graph of closures by borough
closed_df.plot(kind='bar')

In [ ]:
#QUESTION 4: Do certain Boroughs have more rat sightings?
ratcounting=  clean_rats_df.groupby(['Borough']).size().reset_index(name='count')

ratcounting.plot(kind='bar', x='Borough', y='count')
plt.xlabel('Borough')
plt.ylabel('Number of Rat Sightings')
plt.title('Number of Rat Sightings by Borough, 2010-2017')
plt.xticks(rotation=0) 
plt.tight_layout()
plt.show()

In [ ]:
#consider per capita data. Find population and divide the rat sightings by the number of people.
ratcounting = clean_rats_df.groupby(['Borough']).size().reset_index(name='count')

# Your population data
borough_pops = pd.DataFrame({'Borough': ['BRONX', 'BROOKLYN', 'MANHATTAN', 'QUEENS', 'STATEN ISLAND'],
                             'Population': [1471160, 2736074, 1694251, 2405464, 495747]})

# Merge rat sightings data with population data
ratcounting = ratcounting.merge(borough_pops, on='Borough')

# Calculate rat sightings per capita
ratcounting['rat_per_capita'] = ratcounting['count'] / ratcounting['Population']

# Plotting
ratcounting.plot(kind='bar', x='Borough', y='rat_per_capita')
plt.xlabel('Borough')
plt.ylabel('Number of Rat Sightings per Capita')
plt.title('Number of Rat Sightings per Capita by Borough, 2010-2017')
plt.xticks(rotation=0) 
plt.tight_layout()
plt.show()

In [ ]:
#QUESTION 4a: Do certain zip codes have more rat sightings?
rat_burroughs=clean_rats_df[clean_rats_df['Complaint Type']=='Rodent'].groupby('ZIPCODE').size()
rat_burroughs

In [ ]:

#create high, med, low rat sightings

count number of rat sightings per zipcode
categorize each zip codes as high medium and low

In [ ]:
zip_sort=clean_rats_df.groupby("Incident Zip")


In [ ]:
merged_df=pd.merge(clean_grade_df,clean_rats_df, on='Date')



In [ ]:
zip_merge=pd.merge(clean_grade_df,clean_rats_df, on='ZIPCODE')

In [ ]:
#clean_rats_df.columns


In [ ]:
gradecounts= clean_grade_df.groupby(['BORO', 'GRADE']).size().reset_index(name='count')

Boroughs = gradecounts['BORO']
counts = gradecounts['count']
grades = gradecounts['GRADE']

transposecounts = gradecounts.pivot(index='BORO', columns='GRADE', values='count').fillna(0)

transposecounts.plot(kind='bar', stacked=True)
plt.xlabel('Borough')
plt.ylabel('Number of Restaurant Grades')
plt.title('Number of Restaurant Grades by Borough, 2010-2017')
plt.xticks(rotation=0) 
plt.tight_layout()
plt.show()

In [ ]:
clean_grade_df

In [ ]:
ratcounting=  clean_rats_df.groupby(['Borough']).size().reset_index(name='count')

#ratboroughs = ratcounting['Borough']
#ratcounts = ratcounting.groupby('Borough').count()['Complaint Type']


#rattransposecounts = ratcounting.pivot(index='Borough', columns=ratboroughs, values=ratcounts).fillna(0)

ratcounting.plot(kind='bar', x='Borough', y='count')
plt.xlabel('Borough')
plt.ylabel('Number of Rat Sightings')
plt.title('Number of Rat Sightings by Borough, 2010-2017')
plt.xticks(rotation=0) 
plt.tight_layout()
plt.show()

In [ ]:
#consider per capita data. Find population and divide the rat sightings by the number of people.
ratcounting = clean_rats_df.groupby(['Borough']).size().reset_index(name='count')

# Your population data
borough_pops = pd.DataFrame({'Borough': ['BRONX', 'BROOKLYN', 'MANHATTAN', 'QUEENS', 'STATEN ISLAND'],
                             'Population': [1471160, 2736074, 1694251, 2405464, 495747]})

# Merge rat sightings data with population data
ratcounting = ratcounting.merge(borough_pops, on='Borough')

# Calculate rat sightings per capita
ratcounting['rat_per_capita'] = ratcounting['count'] / ratcounting['Population']

# Plotting
ratcounting.plot(kind='bar', x='Borough', y='rat_per_capita')
plt.xlabel('Borough')
plt.ylabel('Number of Rat Sightings per Capita')
plt.title('Number of Rat Sightings per Capita by Borough, 2010-2017')
plt.xticks(rotation=0) 
plt.tight_layout()
plt.show()

In [ ]:
borough_pops=pd.Series([{'BRONX':1471160},
                {'BROOKLYN':2736074},
                {'MANHATTAN':1694251},
                {'QUEENS':2405464},
                {'STATEN ISLAND':495747}])
borough_pops

In [ ]:
dict_lis=[{'BRONX':1471160},
                {'BROOKLYN':2736074},
                {'MANHATTAN':1694251},
                {'QUEENS':2405464},
                {'STATEN ISLAND':495747}]

In [ ]:
[dict_lis]

In [ ]:
borough_count=clean_rats_df['Borough'].value_counts()
borough_count.pop('Unspecified')

In [ ]:
borough_count['BROOKLYN']